# Fetch surronding chucks (N-1, N+1)

This notebook is designed to handle the ingestion of book text (Harry Potter and the Sorcerer's Stone) into an Elasticsearch Cloud instance. It includes partitioning the book text into chapters and chunking the chapter text, which are then ingested into Elasticsearch. The setup utilizes a nested structure, and for each chunk, it stores dense and sparse (ELSER) vector representations along with the text representation.

Searches are performed using dense vector comparisons, sparse vector comparisons, and text search in parallel to demonstrate the power of hybrid search strategies. Additionally, the notebook is configured to retrieve adjacent chunks (n-1 and n+1), allowing for a more contextual understanding of the search results.



## Install required python libraries


In [ ]:
!pip install elasticsearch
!pip install pandas
import json
import time
import urllib.request
import re
import pandas as pd
from transformers import AutoTokenizer, BertTokenizer
from elasticsearch import Elasticsearch, helpers
from google.colab import userdata
import textwrap

# Elasticsearch and Tokenizer Configuration


## Elasticsearch and Tokenizer Configuration

This section sets up the necessary configurations for connecting to Elasticsearch and initializing the tokenizers used for text processing.

### Configuration Details:
1. **Elasticsearch Credentials**:
   - `es_username`: The username for Elasticsearch authentication.
   - `es_password`: The password for Elasticsearch authentication, securely fetched using Google Colab's `userdata` module.
   - `es_cloudid`: The Cloud ID for the Elasticsearch cluster.

2. **Index Settings**:
   - `raw_source_index`: The name of the index for the raw dataset (`harry_potter_dataset-raw`).
   - `index_name`: The name of the enriched dataset index (`harry_potter_dataset_enriched`).
   - `delete_raw_source_index`: A boolean flag indicating whether the raw data index should be deleted before ingesting new data.

3. **Embedding Model**:
   - `dense_embedding_model`: Specifies the model used for generating dense embeddings (`sentence-transformers/all-minilm-l6-v2`).

4. **Tokenizer Initialization**:
   - `bert_tokenizer`: Initializes the BERT tokenizer (`bert-base-uncased`) for English text processing.
   - `e5_tokenizer`: Initializes the Multilingual E5 tokenizer (`intfloat/multilingual-e5-base`) for handling diverse datasets.

5. **Chunking Parameters**:
   - `SEMANTIC_SEARCH_TOKEN_LIMIT`: Sets the token limit for each chunk (500 tokens per chunk, considering space for special tokens).
   - `ELSER_TOKEN_OVERLAP`: Defines the overlap ratio between chunks (default is 0%, customizable for context continuity).

These configurations ensure that the necessary components are properly set up for efficient text processing, indexing, and search operations in Elasticsearch.


In [ ]:
# Elasticsearch and Tokenizer Configuration
es_username = "elastic"
es_password = userdata.get("es_password")
es_cloudid = userdata.get("es_cloudid")

raw_source_index = "harry_potter_dataset-raw"
index_name = "harry_potter_dataset_enriched"

dense_embedding_model = "sentence-transformers__all-minilm-l6-v2"
elser_model_id = ".elser_model_2_linux-x86_64"

bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
e5_tokenizer = AutoTokenizer.from_pretrained("intfloat/multilingual-e5-base")


SEMANTIC_SEARCH_TOKEN_LIMIT = 500
ELSER_TOKEN_OVERLAP = 0.0

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

##Helper Methods/Functions

In [ ]:
def whitespace_tokenize(text):
    return text.split()


def create_es_client():
    return Elasticsearch(cloud_id=es_cloudid, basic_auth=(es_username, es_password))


def manage_index(es, index_name, settings, mappings, delete_index=False):
    if es.indices.exists(index=index_name):
        if delete_index:
            print(f"Index {index_name} exists. Deleting it...")
            es.indices.delete(index=index_name)
            print(f"Index {index_name} deleted!")
        else:
            print(f"Index {index_name} already exists. Skipping creation.")
            return
    es.indices.create(index=index_name, settings=settings, mappings=mappings)
    print(f"Index {index_name} created successfully!")


def generate_actions(df, index_name):
    for _, row in df.iterrows():
        chunks = chunk(row["chapter_full_text"])
        passages = [
            {"text": ch["text"], "chunk_number": ch["chunk_number"]} for ch in chunks
        ]
        doc = {
            "_index": index_name,
            "_source": {
                "book_title": row["book_title"],
                "chapter": row["chapter"],
                "chapter_full_text": row["chapter_full_text"],
                "passages": passages,
            },
        }
        yield doc


def index_dataframe(es, index_name, df, thread_count=1, chunk_size=200):
    print(f"Indexing documents to {index_name}...")
    success_count = 0
    failed_count = 0
    try:
        for success, _ in helpers.parallel_bulk(
            es,
            generate_actions(df, index_name),
            thread_count=thread_count,
            chunk_size=chunk_size,
        ):
            if success:
                success_count += 1
            else:
                failed_count += 1
    except helpers.BulkIndexError as e:
        print("Bulk indexing error:", e)
        for error_detail in e.errors:
            print(error_detail)
    print(f"Successfully indexed {success_count} documents.")
    print(f"Failed to index {failed_count} documents.")


def build_vector(text):
    es = create_es_client()
    docs = [{"text_field": text}]
    response = es.ml.infer_trained_model(model_id=dense_embedding_model, docs=docs)
    return response.get("inference_results", [{}])[0].get("predicted_value", [])


def build_rrf_query(
    embeddings, user_query, rrf_rank_constant, rrf_window_size, debug=False
):
    query = {
        "_source": False,
        "sub_searches": [
            {
                "query": {
                    "nested": {
                        "path": "passages",
                        "query": {"match": {"passages.text": user_query}},
                        "inner_hits": {
                            "name": "text_hits",
                            "size": 1,
                            "_source": ["passages.text", "passages.chunk_number"],
                        },
                    }
                }
            },
            {
                "query": {
                    "nested": {
                        "path": "passages",
                        "query": {
                            "knn": {
                                "query_vector": embeddings,
                                "field": "passages.vector.predicted_value",
                                "num_candidates": 50,
                            }
                        },
                        "inner_hits": {
                            "name": "dense_hit",
                            "size": 1,
                            "_source": ["passages.text", "passages.chunk_number"],
                        },
                    }
                }
            },
            {
                "query": {
                    "nested": {
                        "path": "passages",
                        "query": {
                            "bool": {
                                "should": [
                                    {
                                        "text_expansion": {
                                            "passages.content_embedding.predicted_value": {
                                                "model_id": elser_model_id,
                                                "model_text": user_query,
                                            }
                                        }
                                    }
                                ]
                            }
                        },
                        "inner_hits": {
                            "name": "sparse_hits",
                            "size": 1,
                            "_source": ["passages.text", "passages.chunk_number"],
                        },
                    }
                }
            },
        ],
        "rank": {
            "rrf": {"window_size": rrf_window_size, "rank_constant": rrf_rank_constant}
        },
    }
    if debug:
        print(json.dumps(query, indent=4))
    return query


def build_custom_query(
    query_vector, user_query, knn_boost_factor, text_expansion_boost, debug=False
):
    query = {
        "_source": False,
        "fields": ["chapter"],
        "query": {
            "function_score": {
                "query": {
                    "bool": {
                        "should": [
                            {
                                "nested": {
                                    "path": "passages",
                                    "query": {"match": {"passages.text": user_query}},
                                    "inner_hits": {
                                        "name": "text_hits",
                                        "size": 1,
                                        "_source": [
                                            "passages.text",
                                            "passages.chunk_number",
                                        ],
                                    },
                                }
                            },
                            {
                                "nested": {
                                    "path": "passages",
                                    "query": {
                                        "script_score": {
                                            "query": {
                                                "knn": {
                                                    "field": "passages.vector.predicted_value",
                                                    "query_vector": query_vector,
                                                    "num_candidates": 50,
                                                }
                                            },
                                            "script": {
                                                "source": "Math.log(1 + _score * params.boost_factor)",
                                                "params": {
                                                    "boost_factor": knn_boost_factor
                                                },
                                            },
                                        }
                                    },
                                    "inner_hits": {
                                        "name": "dense_hit",
                                        "size": 1,
                                        "_source": [
                                            "passages.text",
                                            "passages.chunk_number",
                                        ],
                                    },
                                }
                            },
                            {
                                "nested": {
                                    "path": "passages",
                                    "query": {
                                        "script_score": {
                                            "query": {
                                                "bool": {
                                                    "should": [
                                                        {
                                                            "text_expansion": {
                                                                "passages.content_embedding.predicted_value": {
                                                                    "model_id": ".elser_model_2_linux-x86_64",
                                                                    "model_text": user_query,
                                                                }
                                                            }
                                                        }
                                                    ]
                                                }
                                            },
                                            "script": {
                                                "source": "_score * params.boost_factor",
                                                "params": {
                                                    "boost_factor": text_expansion_boost
                                                },
                                            },
                                        }
                                    },
                                    "inner_hits": {
                                        "name": "sparse_hits",
                                        "size": 1,
                                        "_source": [
                                            "passages.text",
                                            "passages.chunk_number",
                                        ],
                                    },
                                }
                            },
                        ]
                    }
                },
                "score_mode": "sum",
                "boost_mode": "sum",
            }
        },
    }
    if debug:
        print(json.dumps(query, indent=4))
    return query


def get_adjacent_chunks_query(doc_id, base_chunk_number, max_chunk_number, debug=False):
    # Determine the chunk numbers to query based on the base_chunk_number
    if base_chunk_number == 1:
        chunk_numbers = [
            base_chunk_number,
            base_chunk_number + 1,
            base_chunk_number + 2,
        ]
    elif base_chunk_number == max_chunk_number:
        chunk_numbers = [
            base_chunk_number,
            base_chunk_number - 1,
            base_chunk_number - 2,
        ]
    else:
        chunk_numbers = [
            base_chunk_number - 1,
            base_chunk_number,
            base_chunk_number + 1,
        ]

    # Construct the query
    query = {
        "_source": False,
        "query": {
            "bool": {
                "must": [
                    {"term": {"_id": doc_id}},
                    {
                        "nested": {
                            "path": "passages",
                            "query": {
                                "bool": {
                                    "should": [
                                        {"term": {"passages.chunk_number": num}}
                                        for num in chunk_numbers
                                    ]
                                }
                            },
                            "inner_hits": {
                                "_source": ["passages.text", "passages.chunk_number"]
                            },
                        }
                    },
                ]
            }
        },
    }

    if debug:
        print(json.dumps(query, indent=4))

    return query


def get_max_chunk_number_query(chapter_number, debug=False):
    # Construct the query
    query = {
        "size": 0,
        "query": {"term": {"chapter": chapter_number}},
        "aggs": {
            "max_chunk_number": {
                "nested": {"path": "passages"},
                "aggs": {"max_chunk": {"max": {"field": "passages.chunk_number"}}},
            }
        },
    }

    if debug:
        print(json.dumps(query, indent=4))

    return query


def print_text_from_results(results):
    if results["hits"]["hits"]:
        for hit in results["hits"]["hits"]:
            if "inner_hits" in hit and "passages" in hit["inner_hits"]:
                nested_hits = hit["inner_hits"]["passages"]["hits"]["hits"]
                for nested_hit in nested_hits:
                    chunk_number = nested_hit["_source"]["chunk_number"]
                    text = nested_hit["_source"]["text"]
                    # print(f"Text from Chunk {chunk_number}: {text}")
                    print(
                        f"\n\nText from Chunk {chunk_number}: {textwrap.fill(first_passage_text, width=200)}"
                    )
    else:
        print("No hits found.")


def chunk(
    text, chunk_size=SEMANTIC_SEARCH_TOKEN_LIMIT, overlap_ratio=ELSER_TOKEN_OVERLAP
):
    step_size = round(chunk_size * (1 - overlap_ratio))
    tokens = bert_tokenizer.encode(text)
    tokens = tokens[1:-1]  # remove special beginning and end tokens
    result = []
    chunk_number = 1
    for i in range(0, len(tokens), step_size):
        end = i + chunk_size
        chunk_text = bert_tokenizer.decode(tokens[i:end])
        result.append({"text": chunk_text, "chunk_number": chunk_number})
        chunk_number += 1
        if end >= len(tokens):
            break
    return result


def check_task_status(es, task_id):
    while True:
        task_response = es.tasks.get(task_id=task_id)
        if task_response["completed"]:
            print("Reindexing complete.")
            break
        else:
            print("Indexing...")
            time.sleep(10)

##Ingest Pipelines

In [ ]:
# Define the ingest pipeline configuration
pipeline_body = {
    "description": "Pipeline for processing book passages",
    "processors": [
        {
            "foreach": {
                "field": "passages",
                "processor": {
                    "inference": {
                        "field_map": {"_ingest._value.text": "text_field"},
                        "model_id": "sentence-transformers__all-minilm-l6-v2",
                        "target_field": "_ingest._value.vector",
                        "on_failure": [
                            {
                                "append": {
                                    "field": "_source._ingest.inference_errors",
                                    "value": [
                                        {
                                            "message": "Processor 'inference' in pipeline 'ml-inference-title-vector' failed with message '{{ _ingest.on_failure_message }}'",
                                            "pipeline": "ml-inference-title-vector",
                                            "timestamp": "{{{ _ingest.timestamp }}}",
                                        }
                                    ],
                                }
                            }
                        ],
                    }
                },
            }
        },
        {
            "foreach": {
                "field": "passages",
                "processor": {
                    "inference": {
                        "field_map": {"_ingest._value.text": "text_field"},
                        "model_id": elser_model_id,
                        "target_field": "_ingest._value.content_embedding",
                        "on_failure": [
                            {
                                "append": {
                                    "field": "_source._ingest.inference_errors",
                                    "value": [
                                        {
                                            "message": "Processor 'inference' in pipeline 'ml-inference-title-vector' failed with message '{{ _ingest.on_failure_message }}'",
                                            "pipeline": "ml-inference-title-vector",
                                            "timestamp": "{{{ _ingest.timestamp }}}",
                                        }
                                    ],
                                }
                            }
                        ],
                    }
                },
            }
        },
    ],
}

# Create or update the pipeline
pipeline_id = "books_dataset_chunker"
es = create_es_client()
es.ingest.put_pipeline(id=pipeline_id, body=pipeline_body)
print(f"Ingest pipeline '{pipeline_id}' created/updated successfully.")

Ingest pipeline 'books_dataset_chunker' created/updated successfully.


##Index Settings

In [ ]:
index_settings = {
    "settings": {
        "number_of_shards": 2,
        "number_of_replicas": 0,
        "default_pipeline": "books_dataset_chunker",
    },
    "mappings": {
        "dynamic": "false",
        "properties": {
            "book_title": {"type": "keyword"},
            "chapter": {"type": "keyword"},
            "chapter_full_text": {"type": "text", "index": False},
            "passages": {
                "type": "nested",
                "properties": {
                    "content_embedding": {
                        "properties": {
                            "is_truncated": {"type": "boolean"},
                            "model_id": {
                                "type": "text",
                                "fields": {
                                    "keyword": {"type": "keyword", "ignore_above": 256}
                                },
                            },
                            "predicted_value": {"type": "sparse_vector"},
                        }
                    },
                    "text": {
                        "type": "text",
                        "fields": {"keyword": {"type": "keyword", "ignore_above": 256}},
                    },
                    "vector": {
                        "properties": {
                            "is_truncated": {"type": "boolean"},
                            "model_id": {
                                "type": "text",
                                "fields": {
                                    "keyword": {"type": "keyword", "ignore_above": 256}
                                },
                            },
                            "predicted_value": {
                                "type": "dense_vector",
                                "dims": 384,
                                "index": True,
                                "similarity": "dot_product",
                            },
                        }
                    },
                    "chunk_number": {"type": "integer"},
                },
            },
        },
    },
}

raw_source_index_settings = {
    "settings": {"number_of_shards": 2, "number_of_replicas": 0},
    "mappings": {
        "dynamic": "false",
        "properties": {
            "book_title": {"type": "keyword"},
            "chapter": {"type": "keyword"},
            "chapter_full_text": {"type": "text", "index": False},
            "passages": {
                "type": "nested",
                "properties": {
                    "text": {
                        "type": "text",
                        "fields": {"keyword": {"type": "keyword", "ignore_above": 256}},
                    },
                    "chunk_number": {"type": "integer"},
                },
            },
        },
    },
}

# Manage indices
manage_index(
    es,
    index_name,
    index_settings["settings"],
    index_settings["mappings"],
    delete_index=True,
)
manage_index(
    es,
    raw_source_index,
    raw_source_index_settings["settings"],
    raw_source_index_settings["mappings"],
    delete_index=True,
)

Index harry_potter_dataset_enriched exists. Deleting it...
Index harry_potter_dataset_enriched deleted!
Index harry_potter_dataset_enriched created successfully!
Index harry_potter_dataset-raw exists. Deleting it...
Index harry_potter_dataset-raw deleted!
Index harry_potter_dataset-raw created successfully!


## Fetch and Process the Book Text

This section downloads the full text of "Harry Potter and the Sorcerer's Stone" from a specified URL and processes it to extract chapters and their titles. The text is then structured into a pandas DataFrame for further analysis and indexing.

### Key Steps:
1. **Download Text**: The book is fetched using `urllib.request` from the provided URL.
2. **Extract Chapters**: The text is split into chapters based on predefined patterns, omitting the text before the first chapter.
3. **Capture Chapter Titles**: Chapter titles are extracted and paired with their respective texts.
4. **Data Structuring**:
   - Convert the list of chapter titles and texts into a DataFrame.
   - Assign sequential numbers to chapters.
   - Add the book title as metadata.
   - Apply a text chunking function to split each chapter into manageable passages.

This prepares the text data for efficient indexing and advanced search operations in Elasticsearch.


In [ ]:
# Fetch and process the book text
potter_book_url = "https://raw.githubusercontent.com/amephraim/nlp/master/texts/J.%20K.%20Rowling%20-%20Harry%20Potter%201%20-%20Sorcerer's%20Stone.txt"
response = urllib.request.urlopen(potter_book_url)
harry_potter_book_text = response.read().decode("utf-8")
chapter_pattern = re.compile(r"CHAPTER [A-Z]+", re.IGNORECASE)
chapters = chapter_pattern.split(harry_potter_book_text)[1:]
chapter_titles = re.findall(chapter_pattern, harry_potter_book_text)
chapters_with_titles = list(zip(chapter_titles, chapters))

print("Total chapters found:", len(chapters))
if chapters_with_titles:
    print("First chapter title:", chapters_with_titles[0][0])
    print("Text sample from first chapter:", chapters_with_titles[0][1][:500])


# Structuring chapters into a DataFrame
df = pd.DataFrame(chapters_with_titles, columns=["chapter_title", "chapter_full_text"])
df["chapter"] = df.index + 1
df["book_title"] = "Harry Potter and the Sorcerer’s Stone"
df["passages"] = df["chapter_full_text"].apply(lambda text: chunk(text))

Total chapters found: 17
First chapter title: CHAPTER ONE
Text sample from first chapter: 

THE BOY WHO LIVED

Mr. and Mrs. Dursley, of number four, Privet Drive, were proud to say
that they were perfectly normal, thank you very much. They were the last
people you'd expect to be involved in anything strange or mysterious,
because they just didn't hold with such nonsense.

Mr. Dursley was the director of a firm called Grunnings, which made
drills. He was a big, beefy man with hardly any neck, although he did
have a very large mustache. Mrs. Dursley was thin and blonde and had
nearly t


Token indices sequence length is longer than the specified maximum sequence length for this model (6535 > 512). Running this sequence through the model will result in indexing errors


## Indexing DataFrame into Elasticsearch

This section uploads the structured data from a pandas DataFrame into a specified Elasticsearch index. The DataFrame contains chapter information from "Harry Potter and the Sorcerer's Stone", including chapter titles, full texts, and additional metadata.

### Key Operation:
- **Index Data**: The `index_dataframe` function is called with the Elasticsearch client, the raw source index name, and the DataFrame as arguments. This operation effectively uploads the data into Elasticsearch, making it searchable and ready for further processing.


In [ ]:
index_dataframe(es, raw_source_index, df)

Indexing documents to harry_potter_dataset-raw...
Successfully indexed 17 documents.
Failed to index 0 documents.


## Asynchronous Reindexing in Elasticsearch

This section initiates an asynchronous reindex operation to transfer data from the raw source index to the enriched index in Elasticsearch. This process runs in the background, allowing other operations to continue without waiting for completion.

### Key Steps:
1. **Start Reindex**: The reindex operation is triggered from the `raw_source_index` to the `index_name`, with `wait_for_completion` set to `False` to allow asynchronous execution.
2. **Retrieve Task ID**: The task ID of the reindex operation is captured and printed for monitoring purposes.
3. **Monitor Progress**: The `check_task_status` function continuously checks the status of the reindex task, providing updates every 10 seconds until the operation is complete.


In [ ]:
# Start the reindex operation asynchronously
response = es.reindex(
    body={"source": {"index": raw_source_index}, "dest": {"index": index_name}},
    wait_for_completion=False,
)
task_id = response["task"]
print("Task ID:", task_id)
check_task_status(es, task_id)

Task ID: _ov-FtHBSkqocXXBG6nu4A:68576798
Indexing...
Reindexing complete.


## Custom Search Query Construction and Execution

This section constructs and executes a custom search query in Elasticsearch, utilizing a hybrid approach combining vector and text-based search methods to enhance search accuracy and relevance. The specific example used is a user query about the "Nimbus 2000".

### Key Steps:
1. **Define User Query**: The user query is specified as "what is a nimbus 2000".
2. **Set Boost Factors**:
   - `knn_boost_factor`: A value to amplify the importance of the vector-based search component.
   - `text_expansion_boost`: A value to modify the weight of the text-based search component.
3. **Build Query**: The `build_custom_query` function constructs the search query, incorporating both dense vector and text expansion components.
4. **Execute Search**: The query is executed against the specified Elasticsearch index.
5. **Identify Relevant Passages**:
   - The search results are analyzed to find the passage with the highest relevance score.
   - The ID and chunk number of the best matching passage are captured and printed.
6. **Fetch Surrounding Chunks**: Constructs and executes a query to retrieve chunks adjacent to the identified passage for broader context. If the matched chunk is the first chunk, fetches n, n+1, and n+2. If the chunk is the last chunk in the chapter, fetches n, n-1, and n-2. For other chunks, fetches n-1, n, and n+1.
7. **Display Results**: Outputs text from the relevant and adjacent passages.

In [ ]:
# Custom Search Query Construction
user_query = "what is a nimbus 2000"


knn_boost_factor = 20
text_expansion_boost = 1
query = build_custom_query(
    build_vector(user_query),
    user_query,
    knn_boost_factor,
    text_expansion_boost,
    debug=False,
)

# Searching and identifying relevant passages
results = es.search(index=index_name, body=query, _source=False)

hit_id = None
chunk_number = None

if results and results.get("hits") and results["hits"].get("hits"):
    highest_score = -1
    best_hit = None
    hit_id = results["hits"]["hits"][0]["_id"]
    chapter_number = results["hits"]["hits"][0]["fields"]["chapter"][0]
    if "inner_hits" in results["hits"]["hits"][0]:
        for hit_type in ["text_hits", "dense_hit", "sparse_hits"]:
            if hit_type in results["hits"]["hits"][0]["inner_hits"]:
                inner_hit = results["hits"]["hits"][0]["inner_hits"][hit_type]["hits"]
                if inner_hit["hits"]:
                    max_score = inner_hit["max_score"]
                    if max_score and max_score > highest_score:
                        highest_score = max_score
                        best_hit = inner_hit["hits"][0]

    if best_hit:
        first_passage_text = best_hit["_source"]["text"]
        chunk_number = best_hit["_source"]["chunk_number"]
        # print(f"Matched Chunk ID: {hit_id}, Chunk Number: {chunk_number}, Text: {first_passage_text}")
        print(
            f"Matched Chunk ID: {hit_id}, Chunk Number: {chunk_number}, Text:\n{textwrap.fill(first_passage_text, width=200)}"
        )
        print(f"\n")
    else:
        print(f"ID: {hit_id}, No relevant passages found.")
else:
    print("No results found.")

print(f"Fetch Surrounding Chunks")
print(f"------------------------")

max_chapter_chunk_result = es.search(
    index=index_name,
    body=get_max_chunk_number_query(chapter_number, debug=False),
    _source=False,
)
max_chunk_number = max_chapter_chunk_result["aggregations"]["max_chunk_number"][
    "max_chunk"
]["value"]

adjacent_chunks_query = get_adjacent_chunks_query(
    hit_id, chunk_number, max_chunk_number, debug=False
)
results = es.search(index=index_name, body=adjacent_chunks_query, _source=False)
print_text_from_results(results)

Matched Chunk ID: wz8m148BbBK3er50L0-W, Chunk Number: 3, Text:
t speaking to us? " said harry. " yes, don't stop now, " said ron, " it's doing us so much good. " hermione marched away with her nose in the air. harry had a lot of trouble keeping his mind on his
lessons that day. it kept wandering up to the dormitory where his new broomstick was lying under his bed, or straying off to the quidditch field where he'd be learning to play that night. he bolted
his dinner that evening without noticing what he was eating, and then rushed upstairs with ron to unwrap the nimbus two thousand at last. " wow, " ron sighed, as the broomstick rolled onto harry's
bedspread. even harry, who knew nothing about the different brooms, thought it looked wonderful. sleek and shiny, with a mahogany handle, it had a long tail of neat, straight twigs and nimbus two
thousand written in gold near the top. as seven o'clock drew nearer, harry left the castle and set off in the dusk toward the quidditch field. held 